In [ ]:
# @title 4. Fetch Route Topology (Google Maps)
# CELL 1 [TAG: parameters]
# ---------------------------------------------------------
# Default parameters (Airflow will OVERWRITE these)
# ---------------------------------------------------------
ORIGIN_ADDRESS = "Universitas Indonesia, Depok"
DESTINATION_ADDRESS = "Monas, Jakarta"
GOOGLE_API_KEY = "YOUR_API_KEY_HERE"
OUTPUT_TOPOLOGY_PATH = "s3://runs/test_run/topology.json"

# MinIO Credentials (DEFAULTS ONLY)
MINIO_ENDPOINT = "http://minio:9000"
MINIO_ACCESS_KEY = "admin"
MINIO_SECRET_KEY = "password123"


In [ ]:
# CELL 2: Imports
import requests
import json
import s3fs


In [ ]:
# CELL 3: MinIO Configuration
# Initialize S3 Filesystem
fs = s3fs.S3FileSystem(
    key=MINIO_ACCESS_KEY,
    secret=MINIO_SECRET_KEY,
    client_kwargs={'endpoint_url': MINIO_ENDPOINT}
)


In [ ]:
# CELL 4: Logic (API Call)
def get_route_topology(origin, dest, api_key):
    # 1. Check for Dummy/Missing Key
    if api_key == "YOUR_API_KEY_HERE" or not api_key:
        print("⚠️ No valid API Key provided. Generating SYNTHETIC topology.")
        return [
            {'condition': 'NORMAL', 'length_m': 2000},
            {'condition': 'TRAFFIC_JAM', 'length_m': 800},
            {'condition': 'NORMAL', 'length_m': 2000}
        ]

    # 2. Prepare Request
    endpoint = "https://routes.googleapis.com/directions/v2:computeRoutes"
    headers = {
        "Content-Type": "application/json",
        "X-Goog-Api-Key": api_key,
        "X-Goog-FieldMask": "routes.distanceMeters,routes.travelAdvisory.speedReadingIntervals"
    }
    body = {
        "origin": {"address": origin},
        "destination": {"address": dest},
        "travelMode": "DRIVE",
        "routingPreference": "TRAFFIC_AWARE_OPTIMAL",
        "extraComputations": ["TRAFFIC_ON_POLYLINE"]
    }

    # 3. Execute Request
    try:
        print(f"Requesting route: {origin} -> {dest}...")
        resp = requests.post(endpoint, headers=headers, json=body, timeout=15)

        if resp.status_code != 200:
            print(f"❌ API Error {resp.status_code}: {resp.text}")
            raise Exception("Google Routes API failed")
        data = resp.json()
        route = data.get("routes", [])[0]
        total_distance = route.get("distanceMeters", 5000)
        intervals = route.get("travelAdvisory", {}).get("speedReadingIntervals", [])

        # 4. Parse Intervals into Topology Blocks
        topology_blocks = []

        # If no specific traffic data (all clear), return one big normal block
        if not intervals:
            return [{'condition': 'NORMAL', 'length_m': total_distance}]

        # Map Google Intervals to our schema
        # Google returns start/end indices of the polyline.
        # We approximate distance based on the proportion of the route
        # (Precise mapping requires decoding the polyline, simplified here for robustness)

        # Simplified approach: We treat the sequence of intervals as the sequence of blocks.
        # Since we don't have exact meters per interval without polyline decoding,
        # we estimate 10 meters per polyline index (rough approximation).

        for interval in intervals:
            speed_type = interval.get('speed', 'NORMAL')
            # Approximate length: (end_idx - start_idx) * avg_segment_length
            # We default to 500m blocks if indices missing
            length = 500
            start = interval.get('startPolylinePointIndex')
            end = interval.get('endPolylinePointIndex')
            if start is not None and end is not None:
                # Rough est: 10 meters per point
                length = (end - start) * 10

            topology_blocks.append({'condition': speed_type, 'length_m': length})

        return topology_blocks
    except Exception as e:
        print(f"❌ Exception during API call: {e}")
        # Fallback
        return [{'condition': 'NORMAL', 'length_m': 5000}]


In [ ]:
# CELL 5: Execution
topology_data = get_route_topology(ORIGIN_ADDRESS, DESTINATION_ADDRESS, GOOGLE_API_KEY)
print(f"Generated Topology: {len(topology_data)} blocks.")
print(topology_data[:3])  # Print sample


In [ ]:
# CELL 6: Save to MinIO
print(f"Saving topology to {OUTPUT_TOPOLOGY_PATH}...")
try:
    with fs.open(OUTPUT_TOPOLOGY_PATH, 'w') as f:
        json.dump(topology_data, f, indent=2)
    print("✅ Topology saved successfully.")
except Exception as e:
    print(f"❌ Failed to save to MinIO: {e}")
    raise
